# STRAW Analysis Procedure
by Matthew Man

20 Aug. 2019

This document will detail the procedure of my analysis of the STRAW data. The results from this analysis are the optical properties of the medium at the time and wavelength of a run (a run refers to a single hdf5 file). Specifically, the attenuation length, absorption length, scattering length, and eta the fraction of molecular scattering, along with respective uncertainties, will be determined. From this, calculating other properties, for example the average cosine of scattering and the effective scattering length, is trivial.

Note: My simulations are wrt POCAM2 only, so any distance calculation will need to be modified to handle POCAM1.

The actual code is in various python files/Jupyter notebooks, where you can also see the output to each cell. Follow this tutorial, with reference to the cource code, to reproduce the analysis.

My github repo https://github.com/matt457/STRAW_analysis

STRAW data can be downloaded at http://tuphecp-haystack.ph.tum.de:10080/ftp/

This document will cover: 
	1. How the photon arrival time distribution is obtained from the data in the hdf5 file
	2. How to calculate (effective) attenuation length from measured time distributions
	3. How to simulate arrival time distributions
	4. How to fit simulations to the data to determine other optical properties


# 1. Measured arrival time distribution

For a complete explanation of this section, Akanksha Katil's documentation should be read because the code for this was originally written by her. Nonetheless the procedure will be documented here. Note that this section will often be quite manual. Unless otherwise specified, the inputs used here should apply in most cases and should not be altered. For reference, this process is also shown in https://github.com/matt457/STRAW_analysis/blob/master/P2%20Violet.ipynb

The clean, run, and residual classes are defined in:

https://github.com/matt457/STRAW_analysis/blob/master/clean.py

https://github.com/matt457/STRAW_analysis/blob/master/run.py

https://github.com/matt457/STRAW_analysis/blob/master/residual.py

WARNING: There are values hardcoded into the files. In clean.py the file_path is hardcoded in to the init method. In residual.py the save_path is hardcoded in the init method, as well as the approximate POCAM interval in ns in the methods init, GetGausPeak, and minimizer. For POCAM1 5000Hz use 200100.71, for POCAM1 2500Hz use 400100.71, for POCAM2 5000Hz use 200101.33, for POCAM2 2500Hz use 400101.33 

It may be worth determining the approximate interval automatically by checking the POCAM frequency in the variable 'values'

In [ ]:
import clean, run, residual

## a) Clean the data

In [ ]:
# Initialize - input the file name
filename = '20190426_085558_UTC_SDOM1_FLASH_POSEIDON1_P2_violet_both_2500Hz_20V_60s_19116085608.hld_up.hdf5'
a = clean.clean(filename)

Output of clean(...) not important

In [ ]:
# Clean the data
(abs_elim_3, rising_0_elim_3, rising_1_elim_3, rising_2_elim_3, rising_3_elim_3,
 falling_0_elim_3, falling_1_elim_3, falling_2_elim_3, falling_3_elim_3, POCAM_num, values,
atstamp, p_jumps, dt_mean, f_r, file_path, SDOM_num, PMT,sub_time_elim_3,sub_id_elim_3) = a.P_S_used()

Output of P_S_used(...) not important, but interesting to visualize times and rates of events

## b) Residuals

In [ ]:
# Initialize residual class
# Note that inputs are dependent on POCAM frequency
# Exact values do not matter

r1 = residual.residual(abs_elim_3,rising_1_elim_3,
                       400080, 400120, # Use POCAM interval +/- 20
                       100,
                       400095, 400105, # Use POCAM interval +/- 5
                       0.0e10,0.6e10,0, 400000, # Optional: specifying the x and y window to visualize residuals
                       file_path, values, POCAM_num, rising_0_elim_3,falling_0_elim_3)

Important output of residual(...) is the first graph produced, a histogram of time differences between events that have an (approximately) gaussian distribution about the actual POCAM frequency. These are referred to as events_in_peak

NOTE: this histogram can be empty, that's not a problem

### b) i) Visualizing residuals

In [ ]:
# Plot 2D histogram of density of residuals over time
gaus_peak = 400100.71 # approximate POCAM interval

# first input is number of bins in histogram, decrease anywhere down to 300 for fainter signals
abs_elim, BinsHist, JumpIndex, xedges, yedges, POCAM_bins, POCAM_diff = r1.HIST2D(500, gaus_peak, SDOM_num)

Residuals are defined as time after POCAM pulse is emitted before it's detected.
Calculated as: residual = (absolute_timestamp)%(POCAM_interval)

The POCAM signal should be bright horizontal lines on the histograms. Take note of roughly where discontinuities in the residuals occur

### b) ii) Exact POCAM frequency - Method 1
Use when events_in_peak looks reasonably gaussian

In [ ]:
# Input absolute time range with no jumps from HIST2D(...)
gaus_peak = r1.minimizer(0.0e10, 0.5e10)

### b) iii) Exact POCAM frequency - Method 2
Will work any time

In [ ]:
# Input absolute time range with no jumps from HIST2D(...)
gaus_peak = r3.GetGausPeak([4.5e10,5.9e10])

### b) iv) Arrival time distribution - Method 1
This method is easiest and works fine when there are many points in events_in_peak

In [ ]:
# Inputs shouldn't need to be modified, and are hard to explain anyway
t_res_all1,  num_events1, noise_events1 = r1.res(gaus_peak, 250, 5, bin_size=1)

Output saved as csv and png. Arbitrarily determined, arrival times should be plotted from -100ns to 200 or 400ns, with the peak always at 0. Currently the analysis depends on 1ns spacing, but this can technically be varied.

Important outputs of res(...) are the last two plots. The 2nd last is the arrival time distribution, and the last one is the recentered time residual of each slice in absolute time, which should line up around 0 with a little variance (black line is mean). If either of these look wrong, try method 2

### b) v) Arrival time distribution - Method 2
Use this method when events_in_peak is sparse

In [ ]:
# rerun HIST2D(...) with exact POCAM interval
abs_elim, BinsHist, JumpIndex, xedges, yedges, POCAM_bins, POCAM_diff = r1.HIST2D(500, gaus_peak, SDOM_num)

# check that correct bins are found (aligned with signal, ie. bright horizontal lines), some error is fine
plt.plot(POCAM_bins, '.')

# Plot arrival time distribution, in small time window, and large one
t_res_all1,num_events1, noise_events1 = r1.calc_res(BinsHist, 10, 2, 1,bin_size=1)

Output (small time window) saved as csv and png

If POCAM_bins are very inaccurate, or if arrival time distribution in small window looks poor, or if the time distr. in large window has multiple (significant) peaks, then try method 3. Generally method 2 only fails when the signal is very faint. 

### b) vi) Arrival time distribution - Method 3
Use this method when the signal is very faint and all else fails. This method stitches together the jumps in residuals by hand

In [ ]:
# rerun HIST2D(...) with exact POCAM interval. Refer to the 2D histograms for the later steps
abs_elim, BinsHist, JumpIndex, xedges, yedges, POCAM_bins, POCAM_diff = r1.HIST2D(500, gaus_peak, SDOM_num)

# select time windows with no jumps (excluding some data is fine)
time_window1 = (abs_elim_3>0.0e10) & (abs_elim_3<1.6e10)
time_window2 = (abs_elim_3>4.7e10) & (abs_elim_3<5.0e10)
time_window3 = abs_elim_3>5.7e10

my_data_all = np.array([])
weights_all = np.array([])

# for a single window at a time,
#  subtract from my_res and reduce plot range until peak is at time 0 with range=(-100,200)
my_slice = (abs_elim_3+rising_1_elim_3)[time_window1] 
weights = r4.weights[time_window1]
my_res = (my_slice%gaus_peak) - 223260
my_data_all = np.append(my_data_all, my_res) # comment out when considering another window
weights_all = np.append(weights_all,weights) # comment out when considering another window

my_slice = (abs_elim_3+rising_1_elim_3)[time_window2]
weights = r4.weights[time_window2]
my_res = (my_slice%gaus_peak) #- 223260-100000+89-500-15
#my_data_all = np.append(my_data_all, my_res)
#weights_all = np.append(weights_all,weights)

my_slice = (abs_elim_3+rising_1_elim_3)[time_window3]
weights = r4.weights[time_window3]
my_res = (my_slice%gaus_peak) #- 223260-100000+89-500-15-123
#my_data_all = np.append(my_data_all, my_res)
#weights_all = np.append(weights_all,weights)

# plot arrival time distribution
fig, ax = plt.subplots(figsize=(10,9))
# vary range to best see peak. eg. range=(-10000,20000) for a rough search, range=(-100,200) for fine search
n,bins,patches = ax.hist(my_data_all,bins=300,log=True, weights=weights_all, range=(-10000,20000))
ax.axvline(color='k')

# save to csv
path = 'Data/POSEIDON1/Measured_arrival_times/'
filename = "['P2'],['SDOM3'],up,violet,['20V'],['2500Hz'].csv"
#with open(path+filename, 'w') as csvfile:
#    writer = csv.writer(csvfile)
#    writer.writerow(bins[:-1])
#    writer.writerow(n)
#csvfile.close()

time_correction_4 = (abs_elim_3[-1]-abs_elim_3[0])/(1.6e10 + (5.0-4.7)*1e10 + (abs_elim_3[-1]-5.7e10))
print(time_correction_4)


With reference to the 2D histograms, select time windows with no jumps (excluding some data is fine). Select as many windows as necessary.

For a single window at a time (comment out other append statements), start with a large plot range, then run the cell, then subtract the peak position from my_res and reduce plot range, then rerun the cell and continue to subtract from my_res to recenter peak to time 0 with final range=(-100,200).

Alternatively, there's a much smarter way of doing this that I've yet to implement. Determine the peak position (median) from a histogram of each my_res slice with an increased number of bins (~80000, we want roughly 1ns bins), and subtract this peak from each my_res slice. Repeat for each slice. It's less manual but either way you still need to define slice ranges (time_window) and recenter the peaks by hand.

When all windows have individually been recentered, uncomment all append statements to plot all slices together, with all peaks at 0.

In some cases, the signal as seen from the 2D histograms is still too faint to discern above noise. In those cases there is nothing else you can do.

## c) Run time

In [ ]:
sDOM1_POSEIDON_run = run.run()
run_time, err_run_time, dead_time_uncert = sDOM1_POSEIDON_run.run_time(atstamp, p_jumps, dt_mean, a.f_0,
                                                    a.f_1-a.r_1,
                                                    a.f_2-a.r_2,
                                                    a.f_3-a.r_3) # reminder: a is instance of the clean class

print('dead_time_uncert: ', dead_time_uncert, '%')

Important outputs are run_time and dead_time_uncert. Currently time corrections are calculated to convert actual run_time to the expected run_time. For example:

actual_run_time = 59s

expected_run_time = 60s

time_correction = 60/59 = 1.016949152542373

Thus when counting photons, counts represents all hits integrated over time span specified by the run, and to compare between runs one must account for both POCAM frequency and run time. Another way of doing this is determining counts/sec.

If method 3 was used, then run_time is defined as the sum of time considered in each time_window.

# 2. Attenuation Length

Refer to https://github.com/matt457/STRAW_analysis/blob/master/Attenuation%20Length.ipynb

Run cells to perform imports, define fucntions photon_counter(...) and function attenuation_length(...)

In [ ]:
# paths to data and to save files
path_meas = "Data/POSEIDON1/violet/Measured_arrival_times/"
path_save = "Data/POSEIDON1/violet/Optimization/"

# 
time_correction = np.array([1.0144305079415528, 0.9885799387876817, 0.57504120677826, 0.3756827648418125])
dead_time_uncert = np.array([0, 0.42392268563089913, 0.7531461586286848, 1.5592730733224076])

peak_photons, total_photons, peak_uncertainty, total_uncertainty = photon_counter(path_meas, time_correction, dead_time_uncert)
att_len, eff_att_len = attenuation_length(path_save, peak_photons, total_photons, peak_uncertainty, total_uncertainty)

Output is data lists for photon counts with percent uncertainties, plots of the linearized intensity-distance relationship, with slope=-1/att_len. The attenuation length and effective attenuation length  with respective systematic uncertainties is calculated. The y-intercept is proportional to ln(N_tot) where N_tot is the total photons emitted. Photon is counted in peak if it lies within (-10,10) of arrival time distribution. 

By default, sDOM5 is excluded due to errant arrival time distribution shapes that give low photon counts. The reason is believed to be saturation of the PMT at short distances from the POCAM. 

Running photon_counter(...) also saves the arrival time distributions with corrections applied, which is used in fitting with simulations.

# 3. Simulations
Refer to https://github.com/matt457/STRAW_analysis/blob/master/Photon%20MC%20simulation.ipynb

Run cells sequentially to perform imports, define probability distributions and helper functions.

As mentioned previously, simulations are wrt POCAM2. It is possible to simulate any distance, but the code must be modified for this (should be trivial to modify).

## a) Running a single simulation
This step is here for reference, and is not strictly necessary to run.

In [ ]:
# Loop over photon_propagation and bin arrival times
# Input: 
#   number of photons to simulate,
#   absorption and scattering lengths in m, 
#   eta the fraction of molecular scattering (float: 0-1), 
#   colour of light (string: 'v'/'violet', 'b'/'blue', 'u'/'uv'),
#   sDOM number (integer: 1-5), 
#   verbosity flag; when False suppress figures,
#   bin width in ns,
#   skip files already in path when True,
#   save results when True.
# Output: photon arrival times (0-150ns, 1ns uniform spacing) and save data as csv

simulation(num_photons, abs_len, scatt_len, eta, colour, sDOM, save_path, verbose=False, bin_size=1, skip_repeat=True, save_results=True)


Output only the execution time, unless verbose=True, in which case also plot the arrival time distribution.

Note: Direct photon time is subtracted from travel times to determine arrival time distribution. However, it is sometimes found that the peak is slightly after the direct photon time. This is accounted for later but ideally the peak should be shifted to 0 either in the function photon_propagation(...) or simulation(...)

## b) Loop over parameter space

In [ ]:
## Loop over parameters and generate popt_array

path = 'Data/Simulation_v2/uv/'

# Parameter selection
num_photons = 100000
colour = 'uv'
sDOM = 5 #1
bin_size = 1 # (ns)

# loop over paramters
scatt_len_list = np.arange(10,26,5)
abs_len_list = np.arange(25,41,5)
eta_list = np.arange(.10,0.36,.05)

# store data
popt_array = np.zeros((len(abs_len_list),len(eta_list),len(scatt_len_list),4))

for i, scatt_len in enumerate(scatt_len_list):
    for j, eta in enumerate(eta_list):
        for k, abs_len in enumerate(abs_len_list):
            arr_time = simulation(num_photons, abs_len, scatt_len, eta, colour, sDOM, path, verbose=False, skip_repeat=True)
            arr_time = arr_time / np.max(arr_time) # normalize

            ind = np.argmax(arr_time) # shift peak to 0
            sim_data_slice = arr_time[ind:-10] # remove edge effects from convolution
            x = np.arange(0,len(sim_data_slice)) * bin_size
            p0 = [10, .02, 3, -.01] # initial parameter guess
            popt, pcov = curve_fit(gaussian_double_exponential,x,sim_data_slice,p0)
            popt_array[k,j,i,:] = popt[:]

np.save(path+'popt_array_sDOM%i_(abs[25,40],eta[.10,.35],scatt[10,25],popt)'%(sDOM),popt_array)   

Simulating 100,000 photons usually takes about 1-3 minutes, but can take anywhere up to 10 minutes when scattering length is very short. Currently I do not reccomend increasing the bin_size, as later steps assume 1ns bin_sizes. All other parameters should be specified by the user, and the code will loop over simulations for every value in scatt_len_list, abs_len_list, eta_list, such that the total number of iterations will be the product of the lengths of these lists. 

Curve fitting with a gaussian-double-exponential is performed on each simulation, and the curve-fit parameters (fwhm, a, k1, k2) are saved in popt_array. The definition of popt_array shows the values along each axis. 5m spacing on scatt/abs length and .05 spacing on eta gives a good balance of simulation time and resolution, but this can be reduced for better resolution. Increasing num_photons will also improve precision of simulations, at the cost of simulation time. 

#### Troubleshooting:

first run plt.semilogy(arr_time) to see the distribution.

Depending on the parameter values, curve_fit may fail. It may be that initial guess p0 is too far from popt, and may need to be adjusted. Try p0 = [10, .002, 3, -.03] when there is little scattering (scatt_len > 25m), and p0 = [10, .02, 3, -.01] when there is a lot of scattering (scatt_len < 25m). If that still doesn't work then play around with p0 till it does. 

It could also be that not enough events occur in highly scattered bins, so arr_time is not smooth enough. The naive way around this is to rerun the simulation in hopes for a smoother curve (this will in fact often work). Otherwise you may have to increase the statistics (eg. try 200,000 photons).

## c) Visualize parameter space

In [ ]:
## Visualize parameter space
## plot 1/chi-squared as function of scatt_len and eta, for each abs_len value

colour = 'uv'
sDOM = 5

save_path = 'Data/Simulation_v2/%s/'%colour
file_sim = 'popt_array_sDOM%i_(abs[25,40],eta[.10,.35],scatt[10,25],popt).npy'%sDOM
popt_array = np.load(save_path+file_sim) #sDOM1

scatt_len_list = np.arange(10,26,5)
abs_len_list = np.arange(25,41,5)
eta_list = np.arange(.10,0.36,.05)

# Get measurement data
path_meas = 'Data/HERA1/%s/Measured_arrival_times/'%colour
file = "['P2'],['SDOM%i'],up,%s,['20V'],['2500Hz'].csv"%(sDOM,colour)

Full source code is deferred here, but important part, the parameter selection, is included. Ensure everything matches up, including the parameter range of popt_array that simulations were run at, the file name of popt_array, the measurement file name and the path etc.

The output is a heatmap of 1/chi_sqr between simulation and measurement for eta vs. scatt_len, at each value of abs_len. Printouts of min(chi_sqr) for each abs_len slice are given. Absolute chi_sqr values here are completely meaningless because the simulations are normalized differently than when the minimization is done, but relative values are still important.

The take-away is that this section should be run to ensure the optimum fit lies somewhere within popt_array (and preferably not at any boundary). Else rerun popt_array at a different range. This visualization will also give a good value to seed the minimizer with. Also best to check each baseline that will be fit over.

# 4. Fit Simulations to Data
Still with reference to https://github.com/matt457/STRAW_analysis/blob/master/Photon%20MC%20simulation.ipynb

## a) Cost function to minimize

In [ ]:
# Poissonian likelihood ratio
def likelihood_ratio(f_obs, f_exp):
    cost = 2*np.sum(f_exp - f_obs + f_obs*np.log(f_obs/f_exp))
    return cost

## b) Simultaneous fit
Only half the function is shown here, where variables are set. Note that almost eveything shown here in the fit function is hardcoded (so inputs are only optical parameters to fit), and as such will need to be specified each time. 

In [ ]:
def simultaneuous_fit_interpolated(abs_len, scatt_len, eta):
    # Parameters
    colour = 'uv'
    save_path = 'Data/Simulation_v2/%s/'%colour
    sDOM_list = [5,1] # specify baselines to fit over
    scatt_len_list = np.arange(10,26,5) # must match parameters lists used in popt_array
    abs_len_list = np.arange(25,41,5)
    eta_list = np.arange(.10,0.36,.05)
    #print(abs_len, scatt_len, eta) # for testing
    
    chi_sqr_total = 0
    
    for sDOM in sDOM_list:
        # measurement
        path_meas = 'Data/MINOS1/%s/Measured_arrival_times/'%colour 
        file = "['P2'],['SDOM%i'],up,%s,['20V'],['2500Hz'],corrected.csv"%(sDOM,colour) 
        my_data = np.genfromtxt(path_meas+file, delimiter=',')
        times = my_data[0][100:(100+139)] # length is abitrary, but will vary with bin_size. 
        counts_slice = my_data[1][100:(100+139)]
        
        # simulation
        file_sim = 'popt_array_sDOM%i_(abs[25,40],eta[.10,.35],scatt[10,25],popt).npy'%sDOM

#### Important notes on the fit function:

A single set of simulations, at a specified baseline and wavelength, and represented by saved values in popt_array, can be used to fit to any run (over time) with the same wavelength and baseline. Temporal shifts shouldn't vary optical parameters greatly. 


times and counts_slice are sliced somewhat arbitrarily. 140ns appeared to be a good range in measurements, after which counts start dropping to a similar level to the background. Measurements start from -100ns with 1ns spacing, hence slicing according to [100:(100+139)]. This would need to change with bin_size. This also determines dofs of likelihood. eg. 139 bins * 2 baselines - 3 parameters - 1 = 274 degrees of freedom. 


Minimizer migrad tends to fail when the likelihood space is not smooth, which is true for simulations of 100,000 photons. Interpolation is the way around that, since linear interpolation will be smooth. The other fit functions simultaneuous_fit_2(...) run simulations at every function call, which is slow and noisy, but theoretically more accurate with larger statistics. Currently I think using simultaneuous_fit_interpolated is the best approach. Just be sure to visualize the parameter space before minimizing, otherwise it's hard to tell if results are sensible (eg. boundaries, local minima, etc).

## c) Minimizer

In [ ]:
m = Minuit(simultaneuous_fit_interpolated, abs_len=30, scatt_len=15, eta=.15, # seed values
           error_abs_len=3, error_scatt_len=2, error_eta=.02, 
           limit_abs_len=(25, 40), limit_scatt_len=(10, 25), limit_eta=(0.10, .35), # set limits
           errordef=.5)


print('Run optimiser')
m.migrad()  # run optimiser
pprint(m.get_param_states())
print('fmin')
pprint(m.get_fmin())

print('Hesse errors')
m.hesse()   # run covariance estimator
print(m.errors)  

Set parameter limits according to those used in popt_array, otherwise interpolation will fail. Seed according to visualization of parameter space, ideally not on a boundary. The error_param values are initial step sizes (rule of thumb is 10% of seed value), and are not vital.

Ensure the printout of fmin has is_valid=True, else results are meaningless. Also ensure that optimal parameters are not on a boundary, else rerun simulations with a wider range. Otherwise you have optimal parameter values and statistical uncertainties, and you're done!
